1.Необходимые библиотеки Python: tkinter, PIL, pandas, astropy, numpy

Изменение кода: измените значение files_name на директорию, где находится папка 'annotation system', например: './annotation system/'

Перед запуском этого кода необходимо загрузить изображения на локальный компьютер, путь сохранения должен быть: files_name + 'fts images'. Для загрузки можно запустить код 'downlowaded.ipynb'

Все данные аннотаций будут сохранены в файле 'annotations.csv'. При повторной аннотации одного и того же изображения будет сохранена последняя версия.

2.Метод использования

Запустите программу, интерфейс отобразит первое изображение для аннотации

Выберите категорию для аннотации (например, "filter")

Нажмите левую кнопку мыши и перетащите, чтобы создать прямоугольник, выделяющий соответствующий элемент

Повторите вышеуказанные шаги для всех шести категорий (при маркировке других категорий предыдущий прямоугольник исчезнет, но информация уже записана)

Нажмите кнопку "Save", чтобы сохранить аннотации текущего изображения

Система автоматически загрузит следующее изображение для аннотации

3.Описание интерфейса

Область отображения изображения: в центре главного окна, показывает текущее изображение FITS

Кнопки категорий: шесть кнопок внизу окна для выбора текущей категории аннотации

Строка состояния: показывает текущую выбранную категорию аннотации

Имя изображения: отображает имя файла текущего аннотируемого изображения

Кнопки операций:

Save: сохранить результаты аннотации текущего изображения

Previous Image: загрузить предыдущее изображение

Next Image: загрузить следующее изображение

4.Замечания

Если текстовая область для определенной категории неясна или прерывиста, не аннотируйте эту категорию для данного изображения

Чтобы исправить ранее аннотированную область нужно еще раз нажать на нужную категорию.

Результаты аннотаций сохраняются в файл annotations.csv только при нажании кнопки Save

Программа показвает позиции ранее аннотированных областей зеленым цветом. При повторном нажании на категорию, зеленая рамка исчезнет и можно нарисовать новую рамку для данной категории.

При следующем запуске программа продолжит работу с последнего аннотированного файла.

Прямоугольник аннотации должен как можно точнее охватывать целевой элемент, однако не следую ставить рамку вплотную к символам внутри элемента.

Данные аннотаций хранятся в виде координатных кортежей в формате (x1, y1, x2, y2)

In [ ]:
import tkinter as tk 
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pandas as pd
import os
import glob
import gzip
import numpy as np
from astropy.io import fits
import shutil

root = tk.Tk()
files_name ='./'# Change to the directory where your annotation system folder is located
images_name = files_name + 'fits_files'  # save directory

# Custom function to convert to JPG file
def fts_to_jpg(image_fts):
    min_val = np.min(image_fts)
    max_val = np.max(image_fts)
    normalized_array = (image_fts - min_val) / (max_val - min_val)  # Normalize
    jpg_scaled_array = normalized_array * 255  # Scale to 0-255 range
    jpg_converted_array = jpg_scaled_array.astype(np.uint8)  # Convert to uint8 integer type

    # Export
    return jpg_converted_array

class ImageAnnotator:
    def __init__(self, root, img_dir, annotation_file):
        self.root = root
        self.img_dir = img_dir
        self.annotation_file = annotation_file
        self.categories = ['filter', 'rell_number', 'date', 'hour', 'minute', 'seconds']
        self.active_category = self.categories[0]
        # Get all fts.gz files
        self.img_files = sorted(glob.glob(os.path.join(self.img_dir, '**', '*.fts.gz'), recursive=True))
        

        # Load annotation file
        if os.path.exists(self.annotation_file):
            self.df = pd.read_csv(self.annotation_file)
        else:
            self.df = pd.DataFrame(columns=['ID', 'FTS.GZ'] + self.categories)

        # Get the last annotated image
        self.img_index = self.load_progress()

        # Create GUI components
        self.create_widgets()

        # Record bounding box data
        self.current_boxes = {cat: None for cat in self.categories}
        self.rect = {cat: None for cat in self.categories}
        self.text = {cat: None for cat in self.categories}
        self.start_x = None
        self.start_y = None

        # Load the first image
        self.load_image()

    def create_widgets(self):
        # Create a Canvas to display the image
        self.canvas = tk.Canvas(self.root, cursor="cross")
        self.canvas.pack()

        # Category selection buttons
        categories = self.categories
        self.category_buttons = {}
        for cat in categories:
            btn = tk.Button(self.root, text=cat, command=lambda c=cat: self.select_category(c))
            btn.pack(side=tk.LEFT)
            self.category_buttons[cat] = btn

        # Display currently selected annotation category
#         self.status_label.config(text=f"Current annotation category: {category}")
        self.status_label = tk.Label(self.root, text=f"Current annotation category: {self.active_category}", anchor="w")
        self.status_label.pack(fill=tk.X)

        # Image file name display
        self.image_name_label = tk.Label(self.root, text="Image Name: ", anchor="w")
        self.image_name_label.pack(fill=tk.X)

        # Save button
        self.save_button = tk.Button(self.root, text='Save', command=self.save_annotation)
        self.save_button.pack(side=tk.LEFT)

        # Image navigation buttons
        self.prev_button = tk.Button(self.root, text='Previous Image', command=self.prev_image)
        self.prev_button.pack(side=tk.LEFT)

        self.next_button = tk.Button(self.root, text='Next Image', command=self.next_image)
        self.next_button.pack(side=tk.LEFT)

        # Bind mouse events
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<B1-Motion>", self.on_mouse_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)

    def load_image(self):
        """Load and display the current image"""
        print(self.img_dir)
        fts_gz_path = self.img_files[self.img_index]
        output_dir = files_name + 'output-fts/'
        os.makedirs(output_dir, exist_ok=True)
        extracted_fits_file = os.path.join(output_dir, os.path.basename(fts_gz_path).replace('.gz', ''))

        # Decompress .fts.gz file
        with gzip.open(fts_gz_path, 'rb') as f_in:
            with open(extracted_fits_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        # Open FITS file and extract data
        hdul = fits.open(extracted_fits_file)
        data = hdul[0].data
        data = np.flipud(data)
        data = fts_to_jpg(data)

        # Convert FITS data to image
        img = Image.fromarray(data)
        img.thumbnail((800, 600))  # Resize image
        self.photo = ImageTk.PhotoImage(img)

        # Display image on Canvas
        self.canvas.config(width=self.photo.width(), height=self.photo.height())
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)

        # Display image name
        self.image_name_label.config(text=f"Image Name: {os.path.basename(fts_gz_path)}")

        # Read previous bounding boxes
        self.load_annotations_for_image()

    def load_annotations_for_image(self):
        """Load and display bounding boxes for the current image"""
        self.clear_boxes()
        img_name = os.path.basename(self.img_files[self.img_index])
        
        current_annotations = self.df.loc[self.df['FTS.GZ'] == img_name]
        if current_annotations.empty:
            return
        
        # Process bounding box coordinates, extract coordinate data from tuples
        current_annotations = current_annotations.dropna(axis=1)
        for cat in self.categories:
            if cat in current_annotations:
                self.create_box(cat, current_annotations[cat].values)

    def clear_boxes(self):
        """Clear all bounding boxes"""
        for rect in self.rect.values():
            if rect is not None:
                self.canvas.delete(rect)

        for text in self.text.values():
            if text is not None:
                self.canvas.delete(text)

        self.rect = {cat: None for cat in self.categories}
        self.text = {cat: None for cat in self.categories}
        self.current_boxes = {cat: None for cat in self.categories}
        
    def select_category(self, category):
        """Select annotation category"""
        self.active_category = category
        if self.rect[category] is not None:
            self.canvas.delete(self.rect[category])
        if self.text[category] is not None:
            self.canvas.delete(self.text[category]) 
        # Update status bar to display current annotation category
        self.status_label.config(text=f"Current annotation category: {category}")

    def on_button_press(self, event):
        """Record initial position on mouse press"""
        if self.active_category is None:
            messagebox.showwarning("Warning", "Please select an annotation category!")
            return

        self.start_x = event.x
        self.start_y = event.y

        cat = self.active_category
        if self.rect[cat]:
            self.canvas.delete(self.rect[cat])
        if self.text[cat]:
            self.canvas.delete(self.text[cat])
        self.rect[cat] = self.canvas.create_rectangle(self.start_x,
                                                      self.start_y,
                                                      self.start_x,
                                                      self.start_y,
                                                      outline="red")

    def on_mouse_drag(self, event):
        """Update rectangle on mouse drag"""
        cur_x, cur_y = (event.x, event.y)
        rect = self.rect[self.active_category]
        self.canvas.coords(rect, self.start_x, self.start_y, cur_x, cur_y)

    def on_button_release(self, event):
        """Record final bounding box on mouse release"""
        end_x, end_y = event.x, event.y
        cat = self.active_category
        self.current_boxes[cat] = (self.start_x, self.start_y, end_x, end_y)
        self.text[cat] = self.canvas.create_text(self.start_x,
                                                 self.start_y - 10,
                                                 anchor=tk.NW,
                                                 text=cat,
                                                 fill="red")

    def prev_image(self):
        """Load previous image"""
        if self.img_index == 0:
            messagebox.showinfo("Info", "No previous images.")
            return
        self.img_index -= 1
        self.load_image()

    def next_image(self):
        """Load next image"""
        if self.img_index == len(self.img_files) - 1:
            messagebox.showinfo("Info", "No next images.")
            return
        self.img_index += 1
        self.load_image()

    def save_annotation(self):
        """Save current image's annotations to CSV"""
        # Check if all categories for the current image are annotated
        missing_categories = [cat for cat, box in self.current_boxes.items() if box is None]
        if missing_categories:
            messagebox.showwarning("Warning", f"Unannotated categories: {', '.join(missing_categories)}")

        # Save annotation data
        img_name = os.path.basename(self.img_files[self.img_index]).replace('.gz', '')
        annotation_data = {
            'ID': img_name,
            'FTS.GZ': os.path.basename(self.img_files[self.img_index])}
        for cat in self.categories:
            annotation_data[cat] = None

        for category, box in self.current_boxes.items():
            if box is not None:
                annotation_data[category] = box  # Save bounding box for each category
        
        if img_name in list(self.df.ID.values):
            for category, box in self.current_boxes.items():
                if box is not None:
                    self.df.loc[self.df.ID == img_name, category] = box
        else:
            self.df = pd.concat([self.df, pd.DataFrame(annotation_data)], ignore_index=True)
        self.df.to_csv(self.annotation_file, index=False)

        # Automatically move to the next image
        self.next_image()

    def load_progress(self):
        """Load progress file, return index of last processed image"""
        if os.path.exists(self.annotation_file):
            # Get the index of the last annotated image from the CSV file
            last_img_name = self.df['FTS.GZ'].iloc[-1] if not self.df.empty else None
            if last_img_name:
                # Get the index of the last annotated image in img_files, and start from the next one
                self.img_index = self.img_files.index(next(f for f in self.img_files if last_img_name in f)) + 1
                if self.img_index >= len(self.img_files):
                    self.img_index = len(self.img_files) - 1  # If out of range, set to the last image
            else:
                self.img_index = 0
        else:
            self.img_index = 0  # If no annotation file, start from the beginning
        return self.img_index

    def create_box(self, name, coords):
        """Draw an annotation box and associate annotation information with it"""
        self.rect[name] = self.canvas.create_rectangle(*coords, outline="green")

        # Display annotation information next to the box
        self.text[name] = self.canvas.create_text(coords[0],
                                                  coords[1] - 10,
                                                  anchor=tk.NW,
                                                  text=name,
                                                  fill="green")

        # Save bounding box data to current_boxes
        self.current_boxes[name] = coords

# Initialize annotation tool
annotator = ImageAnnotator(root, img_dir=images_name,
                           annotation_file=files_name + 'annotations.csv')
root.mainloop()

E:/pythonfile/main-task/annotation system/fts images
